In [35]:
from data_functions import *

import warnings
warnings.filterwarnings('ignore')

import statsmodels.api as sm
import statsmodels.formula.api as smf

import random

In [36]:
for_pres = read('texas_president.xlsx')
for_pres = for_pres.rename(index=str, columns={"count": "county"})
for_pres = for_pres.rename(index=str, columns={"Urbanization Status": "urban", "Border Status": "border"})

In [37]:
for_pres.border.value_counts()

Non-Border    222
Border         32
Name: border, dtype: int64

In [38]:
for_house = read('texas_counties.xlsx')
for_house = for_house.rename(index=str, columns={"rep_number": "district_num", "candidate_name": "dem_candidate", "dem_vote_share": "two_party_vtshare", "Urbanization Status": "urban", "Border Status": "border"})
for_house = for_house[for_house.county != 'ALL COUNTIES']
print(for_house.shape)

(308, 7)


In [39]:
counties = list(for_pres.county)

county_nums = {}
val = 1
for c in counties:
    county_nums[c] = val
    val += 1
    
for_house['county_num'] = for_house.county.apply(lambda x: county_nums[x])
for_house['office'] = 'house'
for_house = for_house[(for_house.two_party_vtshare != 1) & (for_house.two_party_vtshare != 0)]

## COUNTY INFO

In [66]:
county_info = for_pres[['county', 'county_num', 'urban', 'border']]
county_info['urban'] = county_info.urban.apply(lambda x: 1 if x == 'Urban' else 0)
county_info['border'] = county_info.urban.apply(lambda x: 1 if x == 'Border' else 0)

In [67]:
cd = read('demo.csv')
cd = cd.drop(['Unnamed: 0','FIPS','Age'], axis=1)[1:]
cd.County = cd.County.apply(lambda x: x.split(" ")[0])
cd = cd[['County', 'Total', 'Total Female', 'Hispanic Total']]
cd['percent_female'] = cd['Total Female'] / cd['Total']
cd['percent_hispanic'] = cd['Hispanic Total'] / cd['Total']

cd.columns = ['county', 'population', 'Total Female', 'Hispanic Total', 'percent_female','percent_hispanic']
cd = cd[['county', 'population', 'percent_female','percent_hispanic']]
county_info = county_info.merge(cd)

vc = read('vets_and_crimes.xlsx')
vc.county = vc.county.apply(lambda x: x.upper())
county_info = county_info.merge(vc)
county_info = county_info.drop(['crimes_per_100k'], axis=1)

## NEXT INFO

In [ ]:
pres_info = for_pres[['office', 'dem_candidate', 'two_party_vtshare', 'candidate_male', 'candidate_white', 'candidate_hispanic']]
#pres_info = pres_candidates.groupby(['dem_candidate','office']).median()
#pres_info = pres_candidates.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [ ]:
house_info = for_house[['district_num', 'office', 'dem_candidate', 'two_party_vtshare']]
house_info = house_info.groupby(['dem_candidate','office']).median()
house_info = house_info.rename(index=str, columns={"two_party_vtshare": "med_two_party_vtshare"})

In [ ]:
house_info.shape

In [ ]:
house_info.to_csv('house_candidates_to_add_info.csv')

In [ ]:
for_house = for_house.merge(county_info, on=['county', 'county_num'])
for_house.columns

In [ ]:
for_house.border.value_counts()

In [ ]:
demc = for_house[for_house.two_party_vtshare > .5]

In [ ]:
for_house.shape

In [ ]:
mult = for_house.county.value_counts()
mult = list(mult[mult > 1].index)

In [ ]:
mult_df = for_house[for_house.county.isin(mult)]